### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

import mne
import yasa
from scipy.stats import zscore
import plotnine as pn
from plotnine import ggplot, aes, geom_line, geom_rect, scale_x_continuous, labs, theme_classic, theme, element_rect, scale_fill_identity

from utils__helpers_macro import robust_zscore, welch_psd
import utils__config

In [2]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

In [3]:
fif_path = 'Cache/Subject01/Feb02/S01_Feb02_256hz.fif'
potato_path = 'Cache/Subject01/Feb02/S01_potatogram.csv'
bad_epoch_path = 'Cache/Subject01/Feb02/S01_bad_epochs.csv'
bad_channel_path = 'Cache/Subject01/Feb02/S01_bad_channels.csv'

psd_out_path = 'Results/S01_sleep_stage_psd.svg'

Please note that the number of samples must be whole-number divisible by (sampling_freq * epoch_length)

In [4]:
sampling_freq = 256 # Hz
epoch_length = 30 # seconds

### Load Data

In [5]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = False)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove bad channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))
print('Channel count after bad channel removal:', len(raw.ch_names))

# Load the upsampled hypnogram
hypnogram = np.loadtxt(potato_path, delimiter = ',')

C:\Users\lal85\AppData\Local\Temp\2\ipykernel_25124\3029478263.py:1: RuntimeWarning: This filename (Cache/Subject01/Feb02/S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Channel count after bad channel removal: 68


### Epoch Data

Add hypnogram as a channel to the Raw object

In [6]:
# Hypnogram dictionary: 
# (-2) = Unassigned
# (-1) = Artifact
# (0) = Awake
# (1) = N1
# (2) = N2
# (3) = N3
# (4) = REM 

# Re-value sleep stages for incorporation into Epochs object
hypnogram[(hypnogram != 1) & (hypnogram != 2) & (hypnogram != 3)] = 0
hypnogram[(hypnogram == 1) | (hypnogram == 2) | (hypnogram == 3)] = 1

# Create raw object from the hypnogram
hypnogram = hypnogram[np.newaxis, :]

hypno_info = mne.create_info(ch_names = ['hypno'], 
                             sfreq = raw.info['sfreq'], 
                             ch_types = ['misc'])

hypno = mne.io.RawArray(data = hypnogram,
                        info = hypno_info,
                        first_samp = raw.first_samp)

raw.add_channels([hypno], force_update_info = True)

Creating RawArray with float64 data, n_channels=1, n_times=1843574
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.


<Raw | S01_Feb02_256hz.fif, 69 x 1843574 (7201.5 s), ~970.6 MB, data loaded>

Create dummy stim data and an empty stim channel, then fill the channel with the data

In [7]:
# Record the first sample (which is not 0 since the Raw
# file was cropped from the original); you will need this
# to appropriately select the epoch sample number
start = raw.first_samp
step = sampling_freq * epoch_length
stop = raw.last_samp - step

epoch_stim = np.arange(start, stop, step)

# MNE Epochs expects a three column array where the second column
# is a dummy spacer with 0's and the third is an integer indicating
# the ID for the event. So we need to append these to our stim array.
dummy_row = np.zeros(len(epoch_stim))
event_row = np.ones(len(epoch_stim))

epoch_stim = np.vstack((epoch_stim, dummy_row, event_row)).transpose()

# Create a dummy numpy event array and MNE info object
# and use them to create an empty dummy Raw channel
events_info = mne.create_info(ch_names = ['epoch_stim'], 
                              sfreq = raw.info['sfreq'], 
                              ch_types = ['stim'])

empty_events = np.zeros((1, len(raw.times)))

events_channel = mne.io.RawArray(empty_events, events_info)

# Create an event dictionary
event_dictionary = {'epoch_start' : 1}

Creating RawArray with float64 data, n_channels=1, n_times=1843574
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.


Epoch data using dummy stim data in the new channel

In [8]:
# Update the empty stim channel with the formatted epoch start times
raw.add_channels([events_channel], force_update_info = True)
raw.add_events(epoch_stim, 'epoch_stim')

# Find events and create epochs
events = mne.find_events(raw, 
                         stim_channel = 'epoch_stim', 
                         shortest_event = sampling_freq * epoch_length,
                         initial_event = True)

epochs = mne.Epochs(raw, 
                    preload = True, 
                    events = events, 
                    event_id = event_dictionary, 
                    baseline = None,
                    verbose = True,
                    tmin = 0, 
                    tmax = epoch_length)

# Drop the event channel before exporting data
epochs = epochs.drop_channels(['epoch_stim'])

240 events found
Event IDs: [1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 240 events and 7681 original time points ...
0 bad epochs dropped


Select epochs with more than 50% N2/3 sleep and save their sample number start times

In [9]:
# Get epoched hypnogram and get SWS% per epoch
hypochs = epochs.get_data(picks = ['hypno']).squeeze()
hypochs = hypochs.mean(axis = 1).transpose()
hypochs = pd.DataFrame(hypochs, columns = ['hypno_score'])

# Keep epochs with more than 50% of SWS
nopochs = pd.Series(hypochs[hypochs['hypno_score'] <= 0.50].index)
hypochs = pd.Series(hypochs[hypochs['hypno_score'] > 0.50].index)

# Remove hypno channel and get data
epochs = epochs.drop_channels(['hypno'])

# N2/3 sleep
nosleep = epochs[nopochs.tolist()]
nosleep = nosleep.get_data(units = 'uV').transpose(1, 0, 2).reshape(len(epochs.ch_names), -1)

# Non-N2/3 sleep
sleep = epochs[hypochs.tolist()]
sleep = sleep.get_data(units = 'uV').transpose(1, 0, 2).reshape(len(epochs.ch_names), -1)

### Welch PSD

In [10]:
# Welch PSD
tfr_sleep = welch_psd(data = sleep, chan_names = epochs.ch_names, sampling_freq = sampling_freq, n_jobs = 4)
tfr_nosleep = welch_psd(data = nosleep, chan_names = epochs.ch_names, sampling_freq = sampling_freq, n_jobs = 4)

# Average PSD across channels
tfr_sleep = tfr_sleep.groupby('frequency').mean('log_power').reset_index()
tfr_nosleep = tfr_nosleep.groupby('frequency').mean('log_power').reset_index()

# Add sleep stage column
tfr_sleep['stage'] = 'NREM'
tfr_nosleep['stage'] = 'W/REM'

# Merge sleep/nosleep tfr's
tfr = pd.concat([tfr_sleep, tfr_nosleep])

Effective window size : 1.000 (s)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.6s remaining:    2.6s


Effective window size : 1.000 (s)


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished


In [11]:
# Define the data frame for the rectangles
rect_df = pd.DataFrame({
    'xmin': [1, 9],
    'xmax': [4, 15],
    'ymin': [-float('inf'), -float('inf')],
    'ymax': [float('inf'), float('inf')],
    'fill': ['#D3D3D3', '#D3D3D3']  # Light gray color
})

# Static plot with Plotnine
p = (ggplot(tfr)
 + aes(x='frequency', y='log_power', color='stage')
 + geom_rect(data=rect_df, mapping=aes(xmin='xmin', xmax='xmax', ymin='ymin', ymax='ymax', fill='fill'), inherit_aes=False, alpha=0.5)
 + scale_fill_identity()  # Use actual color values in 'fill' column
 + geom_line()
 + scale_x_continuous(expand=(0, 0), limits=(1, 45))
 + labs(x='Frequency (Hz)', y='Log Power (dB)')
 + theme_classic()
 + theme(figure_size=(5, 3), panel_border=element_rect(color='black', size=1))
)

# Save the plot
p.save(filename=psd_out_path, dpi=300)

c:\Users\lal85\AppData\Local\miniconda3\envs\boss\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 5 x 3 in image.
c:\Users\lal85\AppData\Local\miniconda3\envs\boss\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: Results/S01_sleep_stage_psd.svg
c:\Users\lal85\AppData\Local\miniconda3\envs\boss\Lib\site-packages\plotnine\geoms\geom_path.py:98: PlotnineWarning: geom_path: Removed 40 rows containing missing values.
